In [1]:
import pandas as pd
from scipy.sparse import csc_matrix
from scipy.sparse import save_npz

import json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import nltk

import numpy as np

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\袁伟康\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
data_dir = '2022-06-02'

In [4]:
data = pd.read_csv(f'scibert_encoded_papers/needed_paper_metadata_{data_dir}.csv')

In [5]:
data.fillna(' ',inplace=True)

In [6]:
# words interesection
words = []

In [7]:
for idx, row in tqdm(data.iterrows()):
    t_tokens = nltk.word_tokenize(row['title'].lower())
    a_tokens = nltk.word_tokenize(row['abstract'].lower())
    data.at[idx, 'title'] = t_tokens
    data.at[idx, 'abstract'] = a_tokens
    words.extend(t_tokens)
    words.extend(a_tokens)

267948it [05:42, 782.62it/s] 


In [8]:
word_set = set(words)

In [9]:
len(word_set)

609532

In [9]:
# load embedding

In [10]:
word_embs = []
with open('glove.42B.300d.txt','r',encoding='utf-8') as fr:
    for line in tqdm(fr):
        line = line.strip().split(' ')
        word = line[0]
        if word in word_set:
#             embed = map(lambda x:float(x),line[1:])
            word_embs.append(line)

1917495it [00:46, 40936.25it/s]


In [11]:
word_embs = pd.DataFrame(word_embs).set_index(0).astype(float)

In [12]:
final_word_set = set(word_embs.index)

In [13]:
title_embs = []
abstract_embs = []

for idx, row in tqdm(data.iterrows()):
    title_emb = word_embs.loc[list(filter(lambda x:x in final_word_set,row['title']))].mean(0).values
    title_embs.append(title_emb)
    
    abstract_emb = word_embs.loc[list(filter(lambda x:x in final_word_set,row['abstract']))].mean(0).values
    abstract_embs.append(abstract_emb)

267948it [10:01, 445.27it/s]


In [14]:
title_embs_array = np.vstack(title_embs)
title_embs_df = pd.DataFrame(data=title_embs_array,columns=['emb'+str(i) for i in range(300)],index=data['cord_uid']).fillna(0.)

In [15]:
abstract_embs_array = np.vstack(abstract_embs)
abstract_embs_df = pd.DataFrame(data=abstract_embs_array,columns=['emb'+str(i) for i in range(300)],index=data['cord_uid']).fillna(0.)

In [16]:
title_embs_df.to_csv(f'title_embs_df_glove_{data_dir}.csv')
abstract_embs_df.to_csv(f'abstract_embs_df_glove_{data_dir}.csv')